다음의 커널을 필사했습니다.  
https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution/data  
https://www.youtube.com/watch?v=aoo1xrKQXFc&list=PLC_wC_PMBL5PexxXb6UC6pDXODE4xFWzL

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

In [3]:
warnings.simplefilter(action='ignore', category = FutureWarning)

if __name__ == "__main__":  
    pd.set_option('display.max_rows', 60)  
    pd.set_option('display.max_columns', 100)  
    with timer("Pipeline total time"):  
        main(debug= False)  
        
위의 코드로 실행이 되는데 이 노트북에서는 스크립트 형으로 진행하지 않기 때문에 처음부터 설정을 해줍니다.

In [4]:
pd.set_option('display.max_rows', 60)

In [5]:
pd.set_option('display.max_columns', 100)

In [6]:
# main 함수에서 debug = False로 돼있지만 지금은 True이기 때문에 인위적으로 True로 해줍니다.

debug = True
num_rows = 30000 if debug else None

## df = get_train_test(DATA_DIRECTORY, num_rows = num_rows) # 함수 뜯어보기

In [7]:
# GENERAL CONFIGURATIONS
NUM_THRADS = 4
DATA_DIRECTORY = "../input/"
SUBMISSION_SUFIX = "_model2_04"

In [8]:
path = DATA_DIRECTORY
num_rows = num_rows

In [9]:
train_file_path = '../input/home-credit-default-risk/application_train.csv'
test_file_path = '../input/home-credit-default-risk/application_test.csv'

In [10]:
train = pd.read_csv(train_file_path, nrows = num_rows)
test = pd.read_csv(test_file_path, nrows = num_rows)

In [11]:
df = train.append(test)

In [12]:
print(train.shape)

In [13]:
train.head()

In [14]:
train['TARGET'].value_counts().plot.bar()

In [15]:
del train, test

# 메모리 주워 담기
gc.collect

가장 먼저 target값이 imbalance한지 봐야합니다.

# Preprocessing

원래는 EDA를 먼저 들어가야 하지만 여기서는 해당 파트에 대한 설명이 없기 때문에 Preprocessing에 바로 들어갑니다.

In [16]:
# 전체 데이터 프레임을 봤을 때 XNA에는 4명의 사람 밖에 없기 때문에 제거합니다. (아웃라이어로 판단)
df = df[df['CODE_GENDER'] != 'XNA']

In [17]:
# 2천만 이하는 100개 정도 밖에 없기 때문에 이 또한 제거합니다.
df = df[df['AMT_INCOME_TOTAL'] < 20000000]

365243으로 기록된 사람들이 많은데, 이는 의미 없는 숫자로 보입니다. 하지만 이를 제거하면 문제가 생길 수 있습니다. 따라서 일단 nan값을 적용하겠습니다.

In [18]:
df

In [19]:
df[df['DAYS_EMPLOYED'] == 365243].value_counts()

In [20]:
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [21]:
# 핸드폰 바꾼지 0일이라는 것은 null 값을 무언가로 채웠음을 의미하는 것 같습니다. 따라서 nan으로 대체합니다.
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)

In [22]:
# 이들은 binary data입니다. "에, 아니오" 등으로 이루어진 문서
docs = [f for f in df.columns if 'FLAG_DOC' in f]

In [23]:
docs

In [24]:
df[docs].kurtosis(axis=1).hist()

대부분이 특정한 값으로 몰려 있습니다.

In [25]:
# 설문조사에서 "예"를 몇 번 했는지 체크
df['DOCUMENT_COUNT'] = df[docs].sum(axis=1)

# 왜곡된 정도를 컬럼으로 추가
df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1) 

get_age_label 함수의 df['DAYS_BIRTH']를 보면 -살아온 일 수가 기록돼 있습니다.  
여기다 -를 취해 양수로 만들어준 이후 365로 나누면 나이가 나오는데 이를 구간으로 나눠주는 함수를 만듭니다.

In [26]:
df['DAYS_BIRTH'][:5]

continuos한 정보를 categorical한 것으로 바꿔줍니다.

In [27]:
def get_age_label(days_birth):
    """ Return the age group label (int). """
    age_years = -days_birth / 365
    if age_years < 27: return 1
    elif age_years < 40: return 2
    elif age_years < 50: return 3
    elif age_years < 65: return 4
    elif age_years < 99: return 5
    else: return 0

In [28]:
df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x: get_age_label(x))

EXT_SOURCE는 feature importance가 높게 나오는 값들입니다. 이들끼리 서로 interaction할 수 있게 곱해서 새로운 feature를 만들어줍니다.  
현업에서는 설명이 되지 않는 변수는 사용하기 어렵지만 캐글에서는 점수를 높여야 하니 사용하는 경향이 있습니다.

In [29]:
df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']

경진대회에 참여하는 사람들이 해당 feature들 간의 상관관계를 조사해본 결과 2, 1, 3이 좋게 나타났습니다. 따라서 이러한 피처를 추가해줍니다.

In [30]:
df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3

eval() 함수는 np.min, np.var과 같은 통계값을 string 형태로 끼워 넣을 수 있도록 만들어줍니다. 이렇게 하면 다양한 통계값을 직접 손으로 쳐야 하는 상황을 단순하게 만들 수 있다는 장점이 있습니다.

In [31]:
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
    # 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3' 등의 통계값을 구하는 코드
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis = 1)

지금 만드는 피처들은 어느 정도의 도메인 지식이 필요합니다.

In [32]:
# 연금을 받는데 빛은 얼마나 지는지
df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
# 물건을 사는데 빚을 얼마나 지는지
df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']

In [33]:
# Income ratios
df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
 # 일한 햇수에 비해 임금이 얼만지
df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']

In [34]:
# Time ratios
# 살아온 날 대비 얼마나 근로했는지
df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
# 회사에 아이디 만들어진지 얼마나 되는지
df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
# 나이 대비 car를 소유한 일 수
df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']

# do_mean 등 통계값 구하는 함수 뜯어보기

In [35]:
group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']

In [36]:
df, group_cols, counted, agg_name = df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEAN'

In [37]:
# counted가 밑으로 추가됨
group_cols + [counted]

In [38]:
# 목표로 하는 컬럼만 가져옴
df[group_cols + [counted]].head()

df에서 원하는 컬럼만을 가져오고, groupby 이후 통계치를 얻어냅니다. 이후 `reset_index`를 통해 모양을 바로 잡고, 컬럼 이름이 겹치지 않게 rename을 적용해 줍니다.

In [39]:
df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted: agg_name}).head()

위의 작업을 수행한 데이터 프레임을 gp 이름으로 넣어줍니다. 이후 원래 df와 gp를 merge하는데, 두 데이터 프레임이 공통적으로 가지고 있는 group_cols를 기준으로 합쳐줍니다.

In [40]:
def do_mean(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_median(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].median().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_std(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].std().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns = {counted: agg_name})
    df = df.merge(gp, on = group_cols, how = 'left')
    del gp
    gc.collect()
    return df

In [41]:
df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
df = do_mean(df, group, 'AMT_CREDIT', 'CROUP_CREDIT_MEAN')
df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')

## Encode categorical features (LabelEncoder) | 라벨인코더 뜯어보기
df, le_encoded_cols = label_encoder(df, None)  
df = drop_application_columns(df)  
return df

판다스는 문자열을 object라는 자료형으로 나타냅니다. 데이터프레임을 만들 때 dtype을 지정해 주는 것이 아니라면 숫자형을 제외한 나머지는 object로 받아들입니다. 이들을 인코딩 해줍니다.  
참고:  
https://abluesnake.tistory.com/101

In [42]:
for i in df.columns:
    if df[i].dtype == object:
        print(i)

In [43]:
df = df
categorical_columns = None

if not categorical_columns:
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

In [44]:
categorical_columns

In [45]:
for col in categorical_columns:
    break

factorize 함수는 0, 1로 인코딩 된 결과, 그리고 이에 대한 인덱스를 반환합니다.

In [46]:
pd.factorize(df[col])

In [47]:
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

In [48]:
df, le_encoded_cols = label_encoder(df, None)

null importances를 바탕으로 피처를 선택하며 중요도가 낮은 피처는 제거합니다.  
참고:  
https://www.kaggle.com/ogrellier/feature-selection-with-null-importances  
https://databreak.netlify.app/2019-04-21-Null_importance/

In [49]:
for i in df.columns:
    if 'FLAG_DOCUMENT' in i:
        print(i)

In [50]:
def drop_application_columns(df):
    """ Drop features based on permutation feature importance. """
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
        
    # Drop most flag document columns
    for doc_num in [2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
    df.drop(drop_list, axis=1, inplace=True)
    return df

In [51]:
df = drop_application_columns(df)

# get_bureau 함수 뜯어보기

bureau는 당사의 데이터가 아닌 외부 신용평가 기관으로부터 고객의 정보를 가져온 것을 의미합니다.

In [52]:
bureau_path = '../input/home-credit-default-risk/bureau.csv'
bureau = pd.read_csv(bureau_path, nrows=num_rows)

In [53]:
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']

In [54]:
# One hot encoder
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    # categorical_columns 만드는 과정 생략. 위에서 만든 자료 사용했음
    # dummy_na > nan 값을 dummies를 만들 때 포함할 것인지의 여부
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

null data를 nan으로 채우지 않았는데, 그 이유는 LightGBM에서 자동적으로 이러한 값들을 채워주기 때문입니다.

In [55]:
bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category = False)

In [56]:
bureau_balance_path = '../input/home-credit-default-risk/bureau_balance.csv'
bb = pd.read_csv(bureau_balance_path, nrows=num_rows)

In [57]:
bb.head()

breau_balance_STATUS는 loan의 상태를 말해줍니다. (active인지 closed인지 등..)

In [58]:
bb, categorical_cols = one_hot_encoder(bb, nan_as_category = False)

# get_bureau_balance 함수 뜯어보기

`get_bureau_balance` 함수 안에는 `group_and_merge`함수가 있으며 이 안에는 또 `group`함수가 있습니다. `get_bureau_balance`를 이해하기 위해서는 이들이 어떻게 동작하는지 알아야 합니다.

In [59]:
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(bureau_balance_path , nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

In [61]:
# group 코드
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [60]:
# group_and merge 코드
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

In [88]:
categorical_cols

In [62]:
bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()

In [82]:
bb.shape

In [83]:
bb_processed.shape

In [63]:
agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}

In [64]:
df_to_agg = bb
df_to_merge = bb_processed
prefix = ''
aggregations = agg
aggregate_by = 'SK_ID_BUREAU'

In [85]:
agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
agg_df.head()

In [66]:
agg_df.columns.tolist()

In [67]:
# 인덱스를 바꿔주는 코드
agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                          for e in agg_df.columns.tolist()])

In [68]:
agg_df.reset_index().head()

In [90]:
agg_df.shape

위에 코드는 `group` 함수이며 밑에는 `group_and_merge` 함수입니다.

In [96]:
df_to_merge.merge(agg_df, how='left', on=aggregate_by)